In [1]:
!pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.4 MB/s eta 0:00:00


In [2]:
# IMPORT LIBRARIES
import pandas as pd
import numpy as np
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix,classification_report

In [3]:
# LOADING DATASET
df=pd.read_csv('/content/Alphabets_data.csv')

In [4]:
df

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,D,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,C,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,T,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,S,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [6]:
df.info() # checking null values and data dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   letter  20000 non-null  object
 1   xbox    20000 non-null  int64 
 2   ybox    20000 non-null  int64 
 3   width   20000 non-null  int64 
 4   height  20000 non-null  int64 
 5   onpix   20000 non-null  int64 
 6   xbar    20000 non-null  int64 
 7   ybar    20000 non-null  int64 
 8   x2bar   20000 non-null  int64 
 9   y2bar   20000 non-null  int64 
 10  xybar   20000 non-null  int64 
 11  x2ybar  20000 non-null  int64 
 12  xy2bar  20000 non-null  int64 
 13  xedge   20000 non-null  int64 
 14  xedgey  20000 non-null  int64 
 15  yedge   20000 non-null  int64 
 16  yedgex  20000 non-null  int64 
dtypes: int64(16), object(1)
memory usage: 2.6+ MB


In [7]:
df.shape  # checking no of rows and columns

(20000, 17)

In [8]:
df.isnull().sum() # checking for null values

,0
letter,0
xbox,0
ybox,0
width,0
height,0
onpix,0
xbar,0
ybar,0
x2bar,0
y2bar,0


In [9]:
df.duplicated().sum()  # checking for duplicates

1332

In [10]:
df.drop_duplicates(inplace=True)  # dropping duplicates

In [11]:
df.shape

(18668, 17)

In [12]:
x_unscaled=df.drop('letter',axis=1)    # dividing dataset into x and y
y_uncoded=df['letter']

In [13]:
scaler= MinMaxScaler()  # scaling x
x_scaled=scaler.fit_transform(x_unscaled)

In [14]:
x_scaled=pd.DataFrame(x_scaled,columns=x_unscaled.columns)

In [16]:
le=OneHotEncoder()  # applying one hot encoding to y columns
y_en=le.fit_transform(y_uncoded.values.reshape(-1,1))

In [17]:
y_encoded = pd.DataFrame(y_en.toarray())

In [18]:
y_encoded

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
18664,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18665,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# splitting dataset.
x_train,x_test,y_train,y_test=train_test_split(x_scaled,y_encoded,test_size=0.25,random_state=42)

In [20]:
# building model using random parameters
model=Sequential()
model.add(Dense(units=26,activation='relu',input_dim=x_train.shape[1]))
model.add(Dense(units=26,activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
training1=model.fit(x_train,y_train,epochs=30,validation_split=0.25 )  # fitting model

Epoch 1/30
329/329 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0596 - loss: 3.2335 - val_accuracy: 0.2239 - val_loss: 2.9970
Epoch 2/30
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2654 - loss: 2.9101 - val_accuracy: 0.3216 - val_loss: 2.6327
Epoch 3/30
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3706 - loss: 2.5400 - val_accuracy: 0.4493 - val_loss: 2.3178
Epoch 4/30
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4744 - loss: 2.2376 - val_accuracy: 0.4987 - val_loss: 2.0781
Epoch 5/30
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5131 - loss: 2.0217 - val_accuracy: 0.5287 - val_loss: 1.9062
Epoch 6/30
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5471 - loss: 1.8547 - val_accuracy: 0.5427 - val_loss: 1.7898
Epoch 7/30
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5603 - loss: 1.7485 - val_accuracy: 0.5733 - val_loss: 1.6861
Epoch 8/30
329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5861 - loss: 1.6451 - val_accuracy: 0.

In [22]:
model.evaluate(x_test,y_test)  # recieved accuracy of 70%

146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6979 - loss: 1.1394


[1.1316728591918945, 0.6987358331680298]

In [23]:
# using hyperparameter tuning
!pip install scikeras

In [24]:
from sklearn.model_selection import RandomizedSearchCV
from scikeras.wrappers import KerasClassifier

In [25]:
def hyperr(hp):  # creating function
    model = Sequential()
    counter = 0
    for i in range(hp.Int('num_layers', min_value = 1,max_value=10)):
        if counter == 0:
            model.add(Dense(hp.Int('units'+str(i),min_value=1,max_value=128),
                            activation = hp.Choice('activation'+str(i),values = ['relu','elu']),
                            input_dim = 16))
        else: # else block to find no of neurons and activation function
          model.add(Dense(
              hp.Int('units'+str(i),min_value=1,max_value=128),
              activation = hp.Choice('activation'+str(i),values = ['relu','elu'])))
    counter+=1   #Increments the counter to keep track of the layers added.
    model.add(Dense(units = 26,activation=hp.Choice('final_activation',['sigmoid','softmax'])))
    model.compile(optimizer = hp.Choice('optimizer',values = ['RMSprop','sgd','adam']),
                loss = 'categorical_crossentropy',metrics = ['accuracy'])
    return model

In [26]:
tunerr=kt.RandomSearch(hyperr,objective='val_accuracy',max_trials=3)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
tunerr.search(x_train,y_train,epochs=50,validation_data=(x_test,y_test))

Trial 3 Complete [00h 01m 41s]
val_accuracy: 0.03771159052848816

Best val_accuracy So Far: 0.7919434309005737
Total elapsed time: 00h 04m 40s


In [28]:
tunerr.get_best_hyperparameters()[0].values  # getting best parameters

{'num_layers': 3,
 'units0': 33,
 'activation0': 'elu',
 'final_activation': 'sigmoid',
 'optimizer': 'sgd',
 'units1': 84,
 'activation1': 'relu',
 'units2': 125,
 'activation2': 'relu',
 'units3': 91,
 'activation3': 'elu',
 'units4': 81,
 'activation4': 'elu'}

In [29]:
tuned_model=tunerr.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [30]:
tuned_model.fit(x_train,y_train,epochs=100,batch_size=50,validation_data=(x_test,y_test))  # fitting model

Epoch 1/100
281/281 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8016 - loss: 0.6823 - val_accuracy: 0.6968 - val_loss: 1.0055
Epoch 2/100
281/281 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7956 - loss: 0.7002 - val_accuracy: 0.7909 - val_loss: 0.7387
Epoch 3/100
281/281 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8005 - loss: 0.6693 - val_accuracy: 0.7626 - val_loss: 0.8116
Epoch 4/100
281/281 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8036 - loss: 0.6698 - val_accuracy: 0.7701 - val_loss: 0.7724
Epoch 5/100
281/281 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8050 - loss: 0.6590 - val_accuracy: 0.7836 - val_loss: 0.7326
Epoch 6/100
281/281 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8103 - loss: 0.6406 - val_accuracy: 0.8014 - val_loss: 0.6863
Epoch 7/100
281/281 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8158 - loss: 0.6366 - val_accuracy: 0.7765 - val_loss: 0.7443
Epoch 8/100
281/281 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8086 - loss: 0.6420 - val_accu

In [31]:
# @title EVALUATION BEFORE HYPERTUNING
# Get the true labels
y_true = np.argmax(y_test, axis=1)

# Get the predicted labels
y_pred = np.argmax(model.predict(x_test), axis=1)

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred)

print("Model with relu , softmax and adam optimizer ")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Model with relu , softmax and adam optimizer 
Accuracy: 0.6987358045853868
Precision: 0.7116811441337139
Recall: 0.6987358045853868
F1-score: 0.6985786910754833


In [32]:
# @title AFTER HYPERTUNING
# Get the true labels
y_true = np.argmax(y_test, axis=1)

# Get the predicted labels
y_pred = np.argmax(tuned_model.predict(x_test), axis=1)

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred)

print("Model with elu  and sgd optimizer ")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

146/146 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Model with elu  and sgd optimizer 
Accuracy: 0.766016713091922
Precision: 0.7908090920964544
Recall: 0.766016713091922
F1-score: 0.7618571183326979


In [33]:
# Increased in accuracy from 69% to 76% after hyper paramater tuning.

In [34]:
# THANK YOU !!!